## Load sample queries

In [4]:
from adalflow.datasets.big_bench_hard import BigBenchHard
from adalflow.utils.data import subset_dataset

def load_datasets(max_samples: int = None):
    """Load the dataset"""
    train_data = BigBenchHard(split="train")
    val_data = BigBenchHard(split="val")
    test_data = BigBenchHard(split="test")

    # Limit the number of samples
    if max_samples:
        train_data = subset_dataset(train_data, max_samples)
        val_data = subset_dataset(val_data, max_samples)
        test_data = subset_dataset(test_data, max_samples)

    return train_data, val_data, test_data


train, test, val = load_datasets(max_samples = 5)
questions = []
answers = []

for example_instance in train:
    questions.append(example_instance.question)
    answers.append(example_instance.answer)

Saving train split to /teamspace/studios/this_studio/.adalflow/cache_datasets/object_counting/train.csv
Saving val split to /teamspace/studios/this_studio/.adalflow/cache_datasets/object_counting/val.csv
Saving test split to /teamspace/studios/this_studio/.adalflow/cache_datasets/object_counting/test.csv


--2024-10-17 19:55:20--  https://raw.githubusercontent.com/suzgunmirac/BIG-Bench-Hard/main/bbh/object_counting.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35890 (35K) [text/plain]
Saving to: ‘/teamspace/studios/this_studio/.adalflow/cache_datasets/object_counting/object_counting.json’

     0K .......... .......... .......... .....                100% 59.8M=0.001s

2024-10-17 19:55:20 (59.8 MB/s) - ‘/teamspace/studios/this_studio/.adalflow/cache_datasets/object_counting/object_counting.json’ saved [35890/35890]



### Test use case with a simple prompt

In [5]:
from adalflow.components.model_client.ollama_client import OllamaClient
import adalflow as adal
import re

host = "127.0.0.1:11434"

phi3_model = {
        "model_client" : OllamaClient(host=host)
       ,"model_kwargs" : {"model":"phi3:latest"}
}

simple_prompt = r"""
<SYS>You are good at recognizing objects and counting.
Count the number of objects and respond with a numerical answer only.
</SYS>
User question: {{user_query}}
You:
"""
counter_llm = adal.Generator(**phi3_model,template=simple_prompt)

def generate_answer(user_query):
    query = {"user_query": user_query}
    output = counter_llm(query)
    return_count = re.findall(r'\b\d+\b', output.data)[0]
    return return_count

total = 0
correct = 0

for idx, query in enumerate(questions):
    print(query)
    answer = generate_answer(query)
    print(f"predicted {answer} actual {answers[idx]}")
    print("*" * 25)
    total+=1
    if eval(answer) == eval(answers[idx]):
        correct+=1
    
print(f"Accuracy {round(correct/total * 100, 2)}%")

I have an apple, three bananas, a strawberry, a peach, three oranges, a plum, a raspberry, two grapes, a nectarine, and a blackberry. How many fruits do I have?
predicted 10 actual 15
*************************
I have a banana, four strawberries, an apple, two peaches, a plum, a blackberry, and two raspberries. How many fruits do I have?
predicted 10 actual 12
*************************
I have three yams, a lettuce head, an onion, a potato, a cabbage, a carrot, two heads of broccoli, and two stalks of celery. How many vegetables do I have?
predicted 12 actual 12
*************************
I have a chicken, a frog, a mouse, a cat, two pigs, and two rabbits. How many animals do I have?
predicted 5 actual 8
*************************
I have a trombone, a violin, a clarinet, an accordion, four flutes, a trumpet, two drums, and a piano. How many musical instruments do I have?
predicted 10 actual 12
*************************
Accuracy 20.0%
